<a href="https://colab.research.google.com/github/linxiaoxin/DataEngineering/blob/main/colab/Spark%20Query%20Language%20Workshop%20Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
PySpark provides interface used to load DataFrame from external storage systems. We will learn how to read different data format files into DataFrame and write DataFrame back to different data format files using PySpark examples. Lastly, we will learn how to transfer data between JVM and Python processes using Apache Arrow efficiently.

In [6]:
# install pyspark using pip
!pip install --ignore-install -q pyspark
# install findspark using pip
!pip install --ignore-install -q findspark

#from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
import collections
spark = SparkSession.builder.master("local").appName("Ingestion").config('spark.ui.port', '4050').getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 17.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read Driver CSV file
PySpark provides DataFrameReader to load a DataFrame from external storage systems (e.g. file systems, key-value stores, etc). Use SparkSession.read to access this. You can use format(source) to specify the input data source format.  
Using csv("path") or format("csv").load("path") of DataFrameReader, you can read a CSV file into a PySpark DataFrame, These methods take a file path to read from as an argument. When you use format("csv") method, you can also specify the data sources by their fully qualified name, but for built-in sources, you can simply use their short names (csv,json, parquet, jdbc, text e.t.c).
In this example, it shows how to read a single CSV file “people.csv” into DataFrame as well as how to use your own defined schema when read file into DataFrame.


In [5]:
# Read CSV file people.csv
df = spark.read.format('csv') \
                .option("inferSchema","true") \
                .option("header","true") \
                .load("/content/drive/MyDrive/Colab Notebooks/data/BEAD_Rebu_Drivers.csv")

# Show result
df.show()

# Print schema
df.printSchema()


+---+-----------------+-----------+-------------+------+
|Sno|       DriverName|DriverPhone|TaxiIDDriving|Rating|
+---+-----------------+-----------+-------------+------+
|  1|Georgiana Iverson|   38587202|          209|   1.7|
|  2|   Ewell Rolstone|   88675586|          243|   4.9|
|  3|    Pedro Thacker|   94452422|          197|   2.6|
|  4|     Winn Kellard|   81521505|          456|   4.8|
|  5|   Ermin Trounson|   21644415|          372|   4.1|
|  6| Weylin Bernhardi|   89930924|          397|   4.7|
|  7|  Giuseppe Manton|   78503208|          463|   3.3|
|  8| Friedrich De'Ath|   64901517|          264|   4.4|
|  9|  Lauraine Galton|   28736147|          367|   4.7|
| 10|   Debra Willeman|   97189395|          277|   4.6|
| 11| Francene Gavriel|   88137354|           32|   4.9|
| 12|     Eyde Brosini|   34871916|          386|   4.2|
| 13|  Orelia Woolfoot|   28037658|          329|   4.1|
| 14|Christi Middleton|   97577827|          453|   4.8|
| 15|     Jamey Cecely|   48329

DataFrame[]

In [ ]:
driversCount = df.count()
df.show(driversCount, False)

In [ ]:

# Write DataFrame to CSV
df.write.csv("people.csv", header=True, mode="overwrite")
df.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



# Read Taxi JSON
PySpark, a Python API for Apache Spark, handles JSON data efficiently using its powerful data processing capabilities. Some capabilities are:
1. Reading / Writing from/to DataFrame/RDD
2. Handling Nested JSON
3. Performance tuning via caching, partition control and cluster management.

In [ ]:
df = spark.read .option("inferSchema","true") \
                .option("header","true") \
                .option("multiline","True") \
                .json("/content/drive/MyDrive/Colab Notebooks/data/BEAD_Rebu_TaxiCabs.json")
df.show()
df.printSchema()



In [ ]:
df.sort('TaxiNumber').show()

df.sort('TaxiNumber').show()

In [ ]:
df.where("TaxiType = 'Premier' AND TaxiPassengerCapacity= 4").show()

Retrieve all 4 seaters Premier Taxi

In [ ]:
df.select('TaxiNumber', 'TaxiType', 'TaxiColor').where("TaxiType = 'Premier'").show()

# Read Trip DATA CSV

In [24]:
# Read CSV file people.csv
trips = spark.read.format('csv') \
                .option("inferSchema","true") \
                .option("header","true") \
                .load("/content/drive/MyDrive/Colab Notebooks/data/BEAD_Rebu_TripData.csv")

# Show result
df.show()

+---+--------+---+-----------+--------------------+-----------------+-----------------+------------------+------------------------+-------------------+-----------+---------+-------------+--------------------+---------+------------+--------------+
|Sno|    Date|Day|Hour of Day|Trip Start Time HHMM|  Pickup District| DropOff District|Distance Travelled|Trip Duration in Seconds|      Trip End Time|Taxi Number|Taxi Type|Taxi Capacity|Number Of Passengers|Trip Fare|Passenger ID|Passenger Name|
+---+--------+---+-----------+--------------------+-----------------+-----------------+------------------+------------------------+-------------------+-----------+---------+-------------+--------------------+---------+------------+--------------+
|  1|1-Jan-24|Mon|          0| 2024-07-20 00:01:00|        Kew Drive|         Clementi|              25.1|                    3001|2024-07-20 01:50:00|    SHY6685| Standard|            4|                   2|    49.91|       11841|     Ellen Han|
|  2|1-Jan-2

Average distance per trip in January 2024

In [8]:
from pyspark.sql.functions import *

df.agg(avg("Distance Travelled")).first()[0]

11.90267497177833

Total Fare collected by taxi type

In [59]:
df.groupBy('Taxi Type').sum("Trip Fare").where(df['Taxi Type'] == 'Maxi Cab').first()[1]

124899.2199999994

Average Occupancies (no of passenger/passenger capacity) for standtaxi

In [66]:
df.where(df['Taxi Type'] == 'Standard').agg(avg(df['Number Of Passengers']/df['Taxi Capacity'])).first()[0]

0.5380241501943595

Fare collected by day of week from Mon to Sunday

In [75]:
df.cube('Day','Hour of Day').sum('Trip Fare').sort('Day').show()

+----+-----------+------------------+
| Day|Hour of Day|    sum(Trip Fare)|
+----+-----------+------------------+
|NULL|         16| 35219.11999999999|
|NULL|         15| 26466.63000000004|
|NULL|         12| 27622.38999999998|
|NULL|         21|  69611.3300000002|
|NULL|          6| 53495.05000000001|
|NULL|          7| 98835.91999999977|
|NULL|         11|26267.180000000004|
|NULL|          0|          14407.94|
|NULL|          4| 27207.75000000001|
|NULL|         23| 27986.88000000001|
|NULL|          2| 27876.15000000002|
|NULL|         17| 62625.73000000003|
|NULL|          9| 88342.87999999982|
|NULL|          8|115967.63000000008|
|NULL|       NULL|1178257.5900000082|
|NULL|          3|28214.950000000033|
|NULL|         10| 45799.61000000005|
|NULL|         14|25106.230000000003|
|NULL|         18|  81033.4800000002|
|NULL|         13|27023.370000000014|
+----+-----------+------------------+
only showing top 20 rows



Compare total number of trips made by all taxis in weekend vs total

In [20]:


weekdayFare = df.where("DAY not in ('Sat', 'Sun')").agg(sum('Trip Fare')).first()[0]
print(f"weekday fare collected ${weekdayFare}")

weekendFare= df.where("DAY in ('Sat', 'Sun')").agg(sum('Trip Fare')).first()[0]
print(f"weekend fare collected ${weekendFare}")

print(f"Difference: , {weekdayFare - weekendFare}")

weekday fare collected $959439.0700000038
weekend fare collected $218818.52000000043
Difference: ,  740620.55


Import Passenger Data

In [23]:
# Read CSV file people.csv
passenger = spark.read.format('csv') \
                .option("inferSchema","true") \
                .option("header","true") \
                .load("/content/drive/MyDrive/Colab Notebooks/data/BEAD_Rebu_Passengers.csv")

# Show result
df.show()

+---+--------+---+-----------+--------------------+-----------------+-----------------+------------------+------------------------+-------------------+-----------+---------+-------------+--------------------+---------+------------+--------------+
|Sno|    Date|Day|Hour of Day|Trip Start Time HHMM|  Pickup District| DropOff District|Distance Travelled|Trip Duration in Seconds|      Trip End Time|Taxi Number|Taxi Type|Taxi Capacity|Number Of Passengers|Trip Fare|Passenger ID|Passenger Name|
+---+--------+---+-----------+--------------------+-----------------+-----------------+------------------+------------------------+-------------------+-----------+---------+-------------+--------------------+---------+------------+--------------+
|  1|1-Jan-24|Mon|          0| 2024-07-20 00:01:00|        Kew Drive|         Clementi|              25.1|                    3001|2024-07-20 01:50:00|    SHY6685| Standard|            4|                   2|    49.91|       11841|     Ellen Han|
|  2|1-Jan-2

Total fares paid by gold status

In [36]:
#Join tables

goldPassengers = (passenger.where("MemSilvererStGoldtus = 'Gold'")
  .join(trips, passenger.PassengerID == trips["Passenger ID"])
  .agg(sum("Trip Fare"))
  .first()[0]
)

Total = (trips.agg(sum("Trip Fare"))
  .first()[0]
)

print(f"% of fares from Gold passenger {goldPassengers/Total * 100}")

% of fares from Gold passenger 27.735394431025718
